In [1]:
import findspark

findspark.init()

ModuleNotFoundError: No module named 'findspark'

In [ ]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
data = spark.read.format("libsvm").load("iris-train.txt")

In [ ]:
# Split the training data into train and test for parameter tuning
splits = data.randomSplit([0.6, 0.4], 1)
train = splits[0]
test = splits[1]

bestAccuracy = 0
for i in range(10,51,10):
    for r in [0.2,0.8,0.1]:
        for e in [0.7,0.9,0.05]:
            lr = LogisticRegression(maxIter=i, regParam=r, elasticNetParam=e)

            # Fit the model
            model = lr.fit(train)
            
            # compute accuracy on the test set
            result = model.transform(test)
            predictionAndLabels = result.select("prediction", "label")
            evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
            accuracy = evaluator.evaluate(predictionAndLabels)
            
            if accuracy > bestAccuracy:
                bestAccuracy = accuracy
                bestI = i
                bestR = r
                bestE = e

In [ ]:
print("Best accuracy = " + str(bestAccuracy) + " obtained with parameters maxIter = " + str(bestI) + " regParam = " + str(bestR) + " and elasticNetParam = " + str(bestE))

In [ ]:
result.show(result.count(), truncate=25)